In [1]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
def schwefel(x: np.ndarray)-> np.float32 :
    return 418.9829 * len(x) - x.dot(np.sin(np.sqrt(np.abs(x))))

In [31]:
x = np.array([420, 420, 420, 420], dtype=np.float32)

%timeit schwefel(x)

The slowest run took 10.49 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.52 µs per loop


In [30]:
%timeit 418.9829 * len(x) - sum( x*np.sin(np.sqrt(np.abs(x))) for x in x)

The slowest run took 4.55 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 14.7 µs per loop


In [33]:
from math import sin, sqrt
def slow_schwefel(x):
    c = 418.9829
    d = len(x)
    sum = 0
    for v in x:
        sum += v * sin(sqrt(abs(v)))
    
    return c*d - sum

In [35]:
%timeit slow_schwefel(x)

The slowest run took 4.45 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.69 µs per loop


In [3]:
const = 420.968746
inputs = np.array([const, const])
schwefel(inputs)

2.5455132458773733e-05

In [3]:
def generate_population(length=4, lower=-500, upper=500, pop_size=100):
    """Create the initial population"""
    return np.random.uniform(low=lower, high=upper, size=pop_size * length
                     ).reshape([pop_size, length])

In [4]:
def apply_fitness(objective, population):
    """Apply to the objective function to each individaul in population"""

    return np.apply_along_axis(objective, 1, population)    

In [6]:
%%timeit
pop = generate_population(pop_size=1000)
for _ in range(1000):
    fitness = apply_fitness(schwefel, pop)

1 loop, best of 3: 7.18 s per loop


In [5]:
def reproduce(pop, fitness_vals, pop_size=100, survive_prob=0.7, mutation_rate=0.01):
    
    # Shapes
    num_individuals = pop.shape[0]
    num_reproduce = np.int32(num_individuals * survive_prob)

    
    # Best/worst fitness vals 
    sorted_fitness = np.abs(fitness_vals).argsort()
    best = pop[sorted_fitness[:num_reproduce]]
    #worst = pop[sorted_fitness[-num_reproduce:]]
    
    #print(best)

    # Breeders
    #breeders = np.vstack((best, worst))
    breeders = best
    
    # Split into parents
    split_col = pop.shape[1] // 2
    parents_a, parents_b = np.hsplit(breeders, split_col)
    
    # Randomize
    np.random.shuffle(parents_b)
    
    # Mate 
    mated = np.hstack((parents_a, parents_b))

    
    #Mutate the kids
    def rand_val(n):
        '''Returns n random values between [.01, 0.03]'''
        return np.random.randint(1, 3, n) / 100.0
        
    mated_row, mated_col = mated.shape
    mutation = 1.0 - sparse.random(mated_row, mated_col, mutation_rate, data_rvs=rand_val).todense()
    offspring = np.multiply(mutation, mated).A  # matrix -> ndarray
    

    # Fill remaining population with random individuals
    to_fill = pop_size - mated_row
    
    new_individuals = generate_population(length=mated_col, pop_size=to_fill)
    
    

    return np.around(np.vstack((offspring, new_individuals)), decimals=1)
    
    
    

In [40]:
np.random.seed(42)
def evolve():
    EPSILON = 10
    MAX_EPOCHS = 10001
    POP=1000
    MUTATE=0.02
    SURVIVE=0.7
    
    bag = np.zeros([MAX_EPOCHS, 4])
    current_epoch = 0
    best_score = np.inf
    initial_population = generate_population(length=4, pop_size=POP)
    while current_epoch < MAX_EPOCHS:
        
        if current_epoch == 0:
            current_population = initial_population
        
        fit_scores = apply_fitness(schwefel, current_population)
        
        best_individual = current_population[np.abs(fit_scores).argsort()[0]]
        best_fitness = schwefel(best_individual)
        
        if np.abs(best_fitness) < best_score:
            best_score = best_fitness
        
        bag[current_epoch] = best_individual
        current_population = reproduce(current_population, fit_scores, 
                                        pop_size=POP,
                                        mutation_rate=MUTATE,
                                        survive_prob=SURVIVE)
        
        
        
        
        if current_epoch % 500 == 0:
            print('Epoch %d\t Best Fitness %6.4f' % (current_epoch, best_score))
            #print('\tBest Individual: %s\n' % str(best_individual))
        
        
        current_epoch += 1
    
    return bag

results = evolve()

Epoch 0	 Best Fitness 451.6764
Epoch 500	 Best Fitness 2.7466
Epoch 1000	 Best Fitness 1.6412
Epoch 1500	 Best Fitness 1.1870
Epoch 2000	 Best Fitness 1.1870
Epoch 2500	 Best Fitness 1.1870
Epoch 3000	 Best Fitness 1.1447
Epoch 3500	 Best Fitness 1.1447
Epoch 4000	 Best Fitness 1.1447
Epoch 4500	 Best Fitness 1.1447
Epoch 5000	 Best Fitness 1.1447
Epoch 5500	 Best Fitness 0.2771
Epoch 6000	 Best Fitness 0.2771
Epoch 6500	 Best Fitness 0.2771
Epoch 7000	 Best Fitness 0.2771
Epoch 7500	 Best Fitness 0.2771
Epoch 8000	 Best Fitness 0.2771
Epoch 8500	 Best Fitness 0.2771
Epoch 9000	 Best Fitness 0.2771
Epoch 9500	 Best Fitness 0.2771
Epoch 10000	 Best Fitness 0.2771


In [39]:
top_ten = np.apply_along_axis(schwefel, 1, results).argsort()[:10]

results[top_ten]

array([[ 419. ,  419.7,  419. ,  421.2],
       [ 418.6,  422.6,  419. ,  421.9],
       [ 417.9,  419.1,  421.9,  421.8],
       [ 419.6,  424.5,  421.4,  422. ],
       [ 421.8,  417. ,  418.7,  420.5],
       [ 418.7,  423.2,  420.2,  417.6],
       [ 420.9,  425.3,  419.4,  422.1],
       [ 422.9,  418.2,  418.5,  418.6],
       [ 417.7,  423.3,  418.8,  422.5],
       [ 421.7,  422.4,  416.4,  419.9]])

In [21]:
import pandas as pd

In [22]:
df = pd.DataFrame(results[top_ten])
df['output'] = df.apply(schwefel,axis=1)

In [23]:
df

,0,1,2,3,output
0,422.3,420.4,419.7,418.8,1.060448
1,421.3,417.2,424.1,420.1,3.135536
2,426.0,419.5,418.7,421.2,4.123857
3,424.8,421.5,420.9,415.7,5.380632
4,416.8,424.4,421.7,416.2,6.602662
5,416.8,415.9,418.6,418.7,6.774821
6,416.8,415.9,418.6,418.7,6.774821
7,418.3,417.0,425.9,423.7,6.892861
8,424.0,417.9,415.9,424.3,6.979350
9,419.3,428.1,421.6,423.7,7.761693
